# Agents: Adding tools to an agent
---

`Tools` are an abstraction in LangChain that associates a python function with a schema that defines the function's name, description and expected arguments. `Tools` can then be passed to chat models that support `tool calling` allowing the model to request the execution of that function with the required parameters as provided in the schema.

#### What is Tool Calling?

Many AI applications interact directly with humans but what if there was a way for Large Language Models (`LLMs`) to interact directly with systems, such as your databases or external APIs? These systems often have a specific input schema, for example APIs frequently require a payload structure. For this, tool calling enables this functionality:

1. **Tool Creation**: Use the `@tool` decorator to create a tool. A tool is an association between a function and its respective schema.

1. **Tool Binding**: The tool needs to be connected to a model that supports tool calling. This gives model the awareness of the tool and the associated input schema that is required.

1. **Tool Calling**: When appropriate, the model can decide to call a tool and ensure its response conforms to the tool's output schema.

2. **Tool Execution**: The tool can be executed using the arguments provided by the model.

In this short lab, we will see how we can create tools, use tools and use that within our simple agent. We will use the same example from our previous notebook to create a simple agent with certain tools. 

![tool-calling](img/tool_calling.png)

### Set up and imports

In [ ]:
# first, lets import the necessary libraries required to build the agent in this notebook
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import Image, display

### Step 1: Define the Agent State

We will first define the agent state that will remain throughout the operation. This is the state of the graph and the state schema serves as the input for all nodes and edges in the graph.

In [ ]:
class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    itinerary: Optional[str]
    city: str
    user_message: str
    weather_info: Optional[str]
    attractions_info: Optional[str]

### Step 2: Set up language models and prompts

Next, we will set up LLM and a prompt template that will be used by our agent in planning the trip.

In [ ]:
# represents the global variables used across this notebook
BEDROCK_RUNTIME: str = 'bedrock-runtime'
# Model ID used by the agent
AMAZON_NOVA_LITE_MODEL_ID: str = "us.amazon.nova-lite-v1:0"
PROVIDER_ID : str = 'amazon'
# Inference parameters
TEMPERATURE: float = 0.1
MAX_TOKENS: int = 512

In [ ]:
import boto3
import logging
# We are importing this to use any model supported on Amazon Bedrock. In this example
# we will be using the Amazon Nova lite model.
from langchain_aws import ChatBedrockConverse
# This helps checkpoint the memory state of the agent for short term/long term memory
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables.config import RunnableConfig
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
session = boto3.session.Session()
region = session.region_name
logger.info(f"Running this example in region: {region}")

# Initialize the bedrock client placeholder
bedrock_client = boto3.client("bedrock-runtime")

In [ ]:
# Create the llm used by the agent
llm = ChatBedrockConverse(
    model=AMAZON_NOVA_LITE_MODEL_ID,
    provider=PROVIDER_ID, 
    temperature=TEMPERATURE, 
    max_tokens=MAX_TOKENS,
    client=bedrock_client,
)

### Create tools and binding them to an LLM
---

In this section of the notebook, we have now created the `llm` (using `Amazon Nova Lite`). `Amazon Nova Lite` supports function calling. In this case, we can create a function or tools with the `@tool` decorator and associate these tools to the `llm`. The `llm` will then use the tools during invocation and determine which tool to use when a user asks a question.


```python
# Tool creation
tools = [my_tool]
# Tool binding
model_with_tools = model.bind_tools(tools)
# Tool calling 
response = model_with_tools.invoke(user_input)
```

Now, we will create two tools that our `LLM` will have access to while executing the user request:

1. **Search tourist attractions**: We will create a function that does a web search using the `TAVILY` API. This API will be used with the city as requested by the user to look for tourist attractions.

1. **Get weather forecast**: This tool will be used to get insights into the current weather in the city as requested by the user for which they are wanting to create an itinerary.

In [ ]:
# In an ideal scenaio, the tools would call some APIs, such as tavily for web search and the weather API for weather information

In [ ]:
from langchain_core.tools import tool
import json
import os

@tool
def mock_search_tourist_attractions(city: str) -> str:
    """
    Search for tourist attractions in the specified city using a local JSON file.
    
    Args:
        city: The name of the city to search for attractions
        
    Returns:
        A string containing information about tourist attractions in the city
    """
    try:
        # Path to the JSON file containing the attraction data
        file_path = os.path.join("data", "attractions.json")
        
        # Check if the file exists
        if not os.path.exists(file_path):
            return f"Error: Attractions data file not found. Please make sure '{file_path}' exists."
        
        # Load the attractions data from the JSON file
        with open(file_path, 'r') as f:
            attractions_data = json.load(f)
        
        # Check if the city exists in the data
        if city.lower() not in attractions_data:
            return f"No information available for tourist attractions in {city}. Try another city."
        
        # Get the attractions for the specified city
        city_attractions = attractions_data[city.lower()]
        
        # Format the attractions information
        attractions = f"Top boating and swimming attractions in {city}:\n"
        for i, attraction in enumerate(city_attractions, 1):
            attractions += f"{i}. {attraction['title']}: {attraction['description'][:150]}...\n"
        
        return attractions
    
    except Exception as e:
        return f"An error occurred while searching for tourist attractions in {city}: {str(e)}"

@tool
def mock_get_weather_forecast(city: str) -> str:
    """
    Get the current weather forecast for the specified city using a local JSON file.
    
    Args:
        city: The name of the city to get the weather forecast for
        
    Returns:
        A string containing the weather forecast for the city
    """
    try:
        # Path to the JSON file containing the weather data
        file_path = os.path.join("data", "weather.json")
        
        # Check if the file exists
        if not os.path.exists(file_path):
            return f"Error: Weather data file not found. Please make sure '{file_path}' exists."
        
        # Load the weather data from the JSON file
        with open(file_path, 'r') as f:
            weather_data = json.load(f)
        
        # Check if the city exists in the data
        if city.lower() not in weather_data:
            return f"No weather information available for {city}. Try another city."
        
        # Get the weather data for the specified city
        city_weather = weather_data[city.lower()]
        
        # Extract location and current weather information
        location = city_weather["location"]
        current = city_weather["current"]
        
        # Format the weather information
        forecast = f"Current weather in {location['name']}, {location['country']}:\n"
        forecast += f"Local time: {location['localtime']}\n"
        forecast += f"Temperature: {current['temperature']}°C\n"
        forecast += f"Weather: {', '.join(current['weather_descriptions'])}\n"
        forecast += f"Feels like: {current['feelslike']}°C\n"
        forecast += f"Humidity: {current['humidity']}%\n"
        forecast += f"Wind: {current['wind_speed']} km/h, {current['wind_dir']}\n"
        forecast += f"Pressure: {current['pressure']} mb\n"
        forecast += f"Visibility: {current['visibility']} km\n"
        forecast += f"UV Index: {current['uv_index']}\n"
        
        # Add precipitation information if available
        if 'precip' in current:
            forecast += f"Precipitation: {current['precip']} mm\n"
            
        # Add cloud cover information if available
        if 'cloudcover' in current:
            forecast += f"Cloud cover: {current['cloudcover']}%\n"
            
        # Add air quality information if available
        if 'air_quality' in current:
            aq = current['air_quality']
            forecast += "\nAir Quality:\n"
            forecast += f"US EPA Index: {aq['us-epa-index']} "
            
            # Add interpretation of EPA index
            epa_index = aq['us-epa-index']
            if epa_index == 1:
                forecast += "(Good)\n"
            elif epa_index == 2:
                forecast += "(Moderate)\n"
            elif epa_index == 3:
                forecast += "(Unhealthy for sensitive groups)\n"
            elif epa_index == 4:
                forecast += "(Unhealthy)\n"
            elif epa_index == 5:
                forecast += "(Very Unhealthy)\n"
            elif epa_index == 6:
                forecast += "(Hazardous)\n"
            else:
                forecast += "\n"
                
        return forecast
    
    except Exception as e:
        return f"An error occurred while getting the weather forecast for {city}: {str(e)}"

In [ ]:
tools = [mock_search_tourist_attractions, mock_get_weather_forecast]

In [ ]:
from langgraph.prebuilt import create_react_agent

# Create the ReAct agent with the correct prompt
get_realtime_info_react_llm = create_react_agent(
    llm,
    tools=tools
)

In [ ]:
get_realtime_info_react_llm

### Define the nodes and Edges
---

Next, we will be adding nodes, edges and persistent memory to the `StateGraph` before we compile it.

1. user travel plans
1. invoke with Bedrock
1. generate the travel plan for the day
1. ability to add or modify the plan

In [ ]:
# The first node that solely inputs a user message
# The first node that takes in user input
def input_interest(state: PlannerState) -> PlannerState:
    """
    This function processes the user's message.
    """
    # Initialize messages if needed
    if not state.get('messages'): 
        state['messages'] = []
    
    # Return updated state
    return {
        **state
    }

def create_itinerary(state: PlannerState) -> PlannerState:
    try:
        messages = [
            HumanMessage(content=state['user_message'])
        ]
        agent_input = {
            "messages": messages
        }
        
        # Invoke the agent
        result = get_realtime_info_react_llm.invoke(agent_input)
        
        # Extract the output
        if isinstance(result, dict) and "output" in result:
            itinerary = result["output"]
        else:
            itinerary = str(result)
        
        # Return state
        return {
            **state,
            'messages': [
                *state.get('messages', []),
                HumanMessage(content=state['user_message']),
                AIMessage(content=itinerary)
            ],
            'itinerary': itinerary
        }
    except Exception as e:
        logger.error(f"Error creating itinerary: {e}")
        error_message = f"I apologize, but I encountered an error while creating your itinerary: {str(e)}"
        
        # Print the exception for debugging
        import traceback
        traceback.print_exc()
        
        # Return updated state with error
        return {
            **state,
            'messages': [
                *state['messages'],
                HumanMessage(content=state['user_message']),
                AIMessage(content=error_message)
            ],
            'itinerary': error_message
        }


### Create and Compile the Graph
---

In this portion of the notebook, we will create our `LangGraph` workflow and then compile it.

1. First, we will initialize the `StateGraph` with the `State` class that we defined above
1. Then, we add our nodes and edges.
1. We use the `START` Node, a special node that sends user input to the graph, to indicate where to start our graph.
1. The `END` Node is a special node that represents a terminal node.

In [ ]:
# initialize the StateGraph
workflow = StateGraph(PlannerState)
# Next, we will add our nodes to this workflowa
workflow.add_node("input_user_interests", input_interest)
workflow.add_node("create_itinerary", create_itinerary)
workflow.set_entry_point("input_user_interests")
# Next, we will add a direct edge between input interests and create itinerary
workflow.add_edge("input_user_interests", "create_itinerary")
workflow.add_edge("create_itinerary", END)

In [ ]:
# Next, we create a checkpointer which will let us have the graph persist its state
# this is a complete memory for the entire graph
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
# Now that we have compiled the graph, we can view it in a mermaid diagram. You can use Mermaid
# to generate diagrams from markdown-like text.

display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

From the graph above, we can see that from the start point, the user can input the message or their interests for the itinerary. Next, according to the next node, the llm will be invoked with the custom prompt template to create that itinerary. This is a hello world example of a simple agent on `LangGraph`.

### Define the function that runs the graph
---

When we compile the graph, we turn it into a `LangChain` Runnable, which automatically enables calling `.invoke()`, `.stream()` and `.batch()` with your inputs. In the following example, we run `stream()` to invoke the graph with inputs

In [ ]:
def print_stream(result):
    """
    Pretty prints the raw response from a LangGraph workflow result.
    """
    import json
    from pprint import pprint
    
    print("\n" + "=" * 50 + "\n")
    
    # For dictionary results
    if isinstance(result, dict):
        if "messages" in result:
            # Print each message in a readable format
            for i, message in enumerate(result["messages"]):
                # Get message type
                msg_type = type(message).__name__
                print(f"[Message {i+1}] Type: {msg_type}")
                
                # Print content in readable format
                if hasattr(message, "content"):
                    print("\nContent:")
                    if isinstance(message.content, list):
                        for j, part in enumerate(message.content):
                            print(f"\n-- Part {j+1} --")
                            pprint(part)
                    else:
                        print(message.content)
                    
                print("\n" + "-" * 50 + "\n")
        else:
            # Just pretty print the dictionary
            pprint(result)
    
    # For list results
    elif isinstance(result, list):
        for i, item in enumerate(result):
            print(f"\n[Item {i+1}]\n")
            print_stream(item)  # Recursively handle items
    
    # For other types
    else:
        print(f"Type: {type(result)}")
        print("\nContent:")
        print(result)
        
    print("\n" + "=" * 50 + "\n")

In [ ]:
config = {"configurable": {"thread_id": "thread_1"}}
input_data = {"user_message": "I'm interested in visiting Paris for 3 days and I love art, history, and food. Can you suggest an itinerary?"}

# Run the workflow with your input
result = app.invoke(input_data, config=config)

# Display the result
print_stream(result)